In [2]:
#----------GMMA00---------------

import pandas as pd

# Set options
pd.set_option("display.width", 200)
pd.set_option("display.precision", 4)

# Read data from file
file = "/Users/liza/Desktop/Bioinfo Project/amino_acid_genotypes_to_brightness.tsv"
print("\nReading", file)
d = pd.read_csv(file, sep="\t", header=None, skiprows=1, names=["seq", "N", "signal", "sd"])

# Split mutations
mut_list = d["seq"].str.split(":")
mut_list = mut_list.drop(index=0)
# Remove leading "S" from substitutions
mut_list = mut_list.apply(lambda x: [m[1:] for m in x])

# Combine substitutions into a single string
subst = mut_list.apply(lambda x: ":".join(x))
#wieder wie vorher zusammenschreiben, nur ohne S vor jeder Mut
subst[0] = "WT"

# Create data frame
df = pd.DataFrame({"variant": subst, "bright": d["signal"], "n_syn": d["N"], "std": d["sd"]})

# Write data to file
outfile = "amino_acid_genotypes_to_brightness_parsed.tsv"
df.to_csv(outfile, sep="\t", index=False, header=False)

print(f"Dumped {outfile} with {len(subst)} substitutions and {len(mut_list)} variants")

#bereitet die original datei vor, indem es die erste zeile (wt) anders macht und die S entfernt und einen neuen df macht


Reading /Users/liza/Desktop/Bioinfo Project/amino_acid_genotypes_to_brightness.tsv
Dumped amino_acid_genotypes_to_brightness_parsed.tsv with 54025 substitutions and 54024 variants


In [20]:
#----------GMMA01---------------
import numpy as np
# es wird mit dem output des Codes von GMMA00 weitergearbeitet
file = "amino_acid_genotypes_to_brightness_parsed.tsv"

print("Reading", file)

# Read data
try:
    d = pd.read_table(file, header=None)
except FileNotFoundError:
    print("Error: File not found.")
    exit()

# Check if the first row is a label
if isinstance(d.iloc[0, 0], str):
    wt_seq = ""
else:
    wt_seq = d.iloc[0, 0].lower()

# Remove first row (wild type or label)
d = d.iloc[1:]

# Wild-type average log brightness, number of WT measurements, and standard deviation
d.columns = ["seq", "N", "signal", "sd"]
wt = d.iloc[0].copy()
wt["seq"] = ""

print("Making data structures")

# Settings
settings = {}
settings["taa_letters"] = 1

# Per mutant data
def split_mut_list(seq):
    if ":" in seq:
        return seq.split(":")
    else:
        return [seq]

mut_list = d["seq"].fillna('').apply(split_mut_list)

mutant = pd.DataFrame(
    {
        "i": range(1, len(mut_list) + 1),
        "signal": d["signal"],
        "N_sub": [len(sub) for sub in mut_list],
        "N_obs": d["N"],
        "sd_obs": d["sd"],
    }
)
mutant.index = ["mut{:05d}".format(i) for i in range(1, len(mut_list) + 1)]
mutant["subst"] = mut_list.str.join(":")

# Per substitution data
subst_table = pd.Series(mut_list.explode()).value_counts()
nsubst = len(subst_table)
#die Positionen zählen
#subst_table = Positionen der Mutationen
## ?Zählen wieviele Mutationen?

# Make data structure of substitutions
subst = pd.DataFrame({"subst_table": subst_table})
subst["resi"] = 0
subst["taa"] = ""
subst["obs"] = subst["subst_table"]
subst["signal"] = pd.NA
subst.index.name = "Var1"
subst.reset_index(inplace=True)
subst = subst.sort_values("Var1")
subst["i"] = range(1, nsubst + 1)

# Assign residue and taa values
for si in range(1, nsubst + 1):
    m = subst["Var1"].iloc[si - 1]
    if m and m != '':
        subst.at[si, "resi"] = m[1:-settings["taa_letters"]]
        subst.at[si, "taa"] = m[-settings["taa_letters"]:]
#die
# Per residue data
residue = pd.DataFrame(list(wt_seq), columns=["wt"])
residue["subst"] = ""
residue["N_mut"] = pd.NA

# Assign substitutions
for ri in range(1, len(residue) + 1):
    residue.at[ri, "subst"] = subst.loc[subst["resi"] == ri, "taa"].str.cat(sep="")

    si = subst.index[subst["resi"] == ri]
    residue.at[ri, "N_mut"] = subst.loc[si, "obs"].sum()

# Build index translation lists between data frames
res_mut_indices = [mutant.index[mutant["subst"].str.contains(":{}:".format(i))] for i in range(1, len(residue) + 1)]
mut_subst_indices = [subst.index[subst["Var1"].isin(sub)] for sub in mut_list]
subst_mut_indices = []
for si in range(nsubst):
    indices = np.where(mutant["subst"].str.contains(":{}:".format(subst.loc[si, "Var1"])))[0]
    subst_mut_indices.append(indices)


print("")
print("Building subst_mut_indices")
print("done building subst_mut_indices")
nsubst = len(subst_mut_indices)


print("Successfully generated indexed data structures of {} variants carrying {} unique substitutions".format(
    len(mut_list), nsubst))

import pandas as pd

def save_data_to_csv(settings, wt, mut_list, mutant, subst, residue, mut_subst_indices, subst_mut_indices, res_mut_indices, file="gmma_structured.csv"):
    data = {
        "settings": pd.DataFrame(settings, index=[0]),
        "wt": wt,
        "mut_list": pd.DataFrame({"seq": mut_list}),
        "mutant": mutant,
        "subst": subst,
        "residue": residue,
        "mut_subst_indices": pd.DataFrame({"mut_subst_indices": mut_subst_indices}),
        "subst_mut_indices": pd.DataFrame({"subst_mut_indices": subst_mut_indices}),
        "res_mut_indices": pd.DataFrame({"res_mut_indices": res_mut_indices})
    }

    with open(file, "w") as f:
        for key, value in data.items():
            f.write(f"---{key}---\n")
            value.to_csv(f, index=False)
            f.write("\n")

    print(f"Saved {file}")

# Beispielaufruf
save_data_to_csv(settings, wt, mut_list, mutant, subst, residue, mut_subst_indices, subst_mut_indices, res_mut_indices, file="gmma_structured.csv")

#hat glaube ich geklappt: 54024 (im csv file: eine Zeile head und eine wt) variants mit 1879 (stimmt) substitutions

Reading amino_acid_genotypes_to_brightness_parsed.tsv
Making data structures

Building subst_mut_indices
done building subst_mut_indices
Successfully generated indexed data structures of 54024 variants carrying 1879 unique substitutions
Saved gmma_structured.csv


In [21]:
#----------GMMA02---------------
import sys
import numpy as np
import pandas as pd

# Set R environment options
options = {
    "width": 120,
    "digits": 3,
    "stringsAsFactors": False
}

# Analysis settings----------------------
RT = 1.985*300/1000.0          # RT value
unit_RT = "kcal/mol (at 300 K)" # RT unit
dG_wt = -2.8
# dG_wt_input = dG_wt

#--------------------------------------

# Load data from file
data = pd.read_csv("gmma_structured.csv", sep='\t')


# Assign number of rows
nmut = data["mutant"].shape[0]
nsubst = data["subst"].shape[0]
nres = data["residue"].shape[0]


# Initialize cp list
cp = {"dG_wt": dG_wt}

# Assign active column in mutant data frame
cp["B_mid"] = data["mutant"]["signal"].quantile(0.5)
data["mutant"]["active"] = np.where(data["mutant"]["signal"] < cp["B_mid"], 0, 1)

# Assign active and inactive counts in subst data frame
active_cut = data["mutant"]["active"].mean() + 2 * data["mutant"]["active"].std()
inactive_cut = active_cut
data["subst"]["active"] = np.where(data["mutant"]["active"] == 1, 1, 0)
data["subst"]["inactive"] = np.where(data["mutant"]["active"] == 0, 1, 0)
active_count = data["subst"].groupby("subst")["active"].sum()
inactive_count = data["subst"].groupby("subst")["inactive"].sum()
data["subst"] = pd.merge(data["subst"], active_count, on="subst")
data["subst"] = pd.merge(data["subst"], inactive_count, on="subst")

# Assign active and inactive counts in residue data frame
data["residue"]["active"] = data["subst"].groupby("residue")["active"].sum()
data["residue"]["inactive"] = data["subst"].groupby("residue")["inactive"].sum()

# Clean data for initial fitting
data["mutant"]["init"] = data["mutant"]["signal"]
data["mutant"]["init_m"] = data["mutant"]["mutant"]
data["subst"]["init_m"] = data["subst"]["mutant"]
data["subst"]["init"] = data["subst"]["signal"]

# Define exclude_data function
def exclude_data(data, condition, message):
    count = data.shape[0]
    data = data[~condition]
    excluded_count = count - data.shape[0]
    print(f"{excluded_count} {message} excluded.")
    return data, excluded_count

# Exclude specific data based on conditions
data["subst"], excluded_subst = exclude_data(data["subst"], data["subst"]["si_nonsense"], "substitutions with nonsense mutants")
data["mutant"], excluded_mutants = exclude_data(data["mutant"], data["mutant"]["si_lowobs"], "low observed mutants")
data["subst"], excluded_subst_lowact = exclude_data(data["subst"], data["subst"]["si_lowact"], "substitutions with low activity")

# Calculate probability of IFS
prob_IFS = data["subst"]["inactive"] / (data["subst"]["inactive"] + data["subst"]["active"])
data["subst"]["log_prob_IFS"] = np.log10(prob_IFS)

# Calculate probability of IF
prob_IF = data["subst"]["active"] / (data["subst"]["inactive"] + data["subst"]["active"])
data["subst"]["log_prob_IF"] = np.log10(prob_IF)

# Fit data for linear regression
fit = np.polyfit(data["subst"]["init"], data["subst"]["signal"], 1, full=True)

# Calculate dG_wt using linear regression
dG_wt_lr = fit[0][1]

# Initialize lists for statistics
cp["nsubst_lowact"] = []
cp["nsubst_highact"] = []
cp["subst_highact"] = []
cp["dG_wt_corr"] = []

# Iteratively adjust dG_wt and perform calculations
for i in range(4):
    # Identify low and high active substitutions
    low_act_cut = data["subst"]["active"].quantile(0.25)
    high_act_cut = data["subst"]["active"].quantile(0.75)
    low_act_subst = data["subst"][data["subst"]["active"] < low_act_cut]
    high_act_subst = data["subst"][data["subst"]["active"] > high_act_cut]

    # Calculate average signal of low and high active substitutions
    low_act_mean = low_act_subst["signal"].mean()
    high_act_mean = high_act_subst["signal"].mean()

    # Calculate difference in average signal
    diff_mean = high_act_mean - low_act_mean

    # Update dG_wt
    dG_wt = dG_wt_lr + diff_mean

    # Exclude substitutions with low activity
    data["subst"], excluded_subst_lowact = exclude_data(data["subst"], data["subst"]["signal"] < dG_wt, "substitutions with low activity")

    # Calculate linear regression with updated dG_wt
    fit = np.polyfit(data["subst"]["init"], data["subst"]["signal"], 1, full=True)
    dG_wt_lr = fit[0][1]

    # Calculate correlation coefficient
    corr_coef = np.corrcoef(data["subst"]["signal"], data["subst"]["init"])[0, 1]

    # Update lists with statistics
    cp["nsubst_lowact"].append(excluded_subst_lowact)
    cp["nsubst_highact"].append(high_act_subst.shape[0])
    cp["subst_highact"].append(high_act_subst["subst"].tolist())
    cp["dG_wt_corr"].append(corr_coef)

# Print final results
print("Final results:")
print("dG_wt:", dG_wt)
print("Correlation coefficient:", corr_coef)


KeyError: 'mutant'